In [10]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import time
import Levenshtein as lev

In [11]:
forbes_list_2018 = pd.read_csv("file:///Users/tbro/PycharmProjects/corpedia/Forbes_2000_2018.csv")

In [12]:
forbes_list_2018 = forbes_list_2018[["rank", "name", "country"]]
forbes_list_2018 = forbes_list_2018.sort_values(by="rank")
forbes_list_2018.set_index("rank", drop= True)

,name,country
rank,,
1,ICBC,China
2,China Construction Bank,China
3,JPMorgan Chase,United States
4,Berkshire Hathaway,United States
5,Agricultural Bank of China,China
6,Bank of America,United States
7,Wells Fargo,United States
8,Apple,United States
9,Bank of China,China


In [31]:
def QuerySparql(query):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()
    return results['results']['bindings']

In [32]:
def GenQuery(raw_name): 
    result= """SELECT DISTINCT ?item ?itemLabel ?alias WHERE {?item wdt:P414 ?stock; rdfs:label ?itemLabel; skos:altLabel ?alias FILTER (lang(?itemLabel) = 'en'). FILTER (LANG (?alias) = "en"). FILTER regex (?alias, '^.*""" + raw_name + ".*$').}"
    return result

In [33]:
def GetWikiIDs(query_list, raw_name):
    wiki_id = query_list["item"]["value"].replace("http://www.wikidata.org/entity/","")
    match_name = query_list["itemLabel"]["value"]
    alias = query_list["alias"]["value"]
    if (lev.ratio(match_name.lower(),raw_name.lower())<0.5 or lev.ratio(alias.lower(),raw_name.lower())<0.5):
        return wiki_id, match_name
    else:
        return NA

In [41]:
def GetWikiIDsFromRawName (raw_name):
    query = GenQuery(raw_name)
    result_list = QuerySparql(query)
    result = list(map(GetWikiIDs, result_list,raw_name))
    time.sleep(6)
    return(result)

In [35]:
raw_name="ICBC"
query = GenQuery(raw_name)
result_list = QuerySparql(query)
result = list(map(GetWikiIDs, result_list, raw_name))

In [36]:
print(result)

[('Q26463', 'Industrial and Commercial Bank of China')]

In [42]:
Forbes_2000_raw_names = pd.Series(forbes_list_2018.iloc[10:20,1])

In [43]:
Forbes_2000_raw_names.index = Forbes_2000_raw_names

In [44]:
Forbes_2000_IDs = Forbes_2000_raw_names.apply(GetWikiIDsFromRawName)

In [45]:
Forbes_2000_raw_names

name
Royal Dutch Shell              Royal Dutch Shell
Toyota Motor                        Toyota Motor
ExxonMobil                            ExxonMobil
Samsung Electronics          Samsung Electronics
AT&T                                        AT&T
Volkswagen Group                Volkswagen Group
HSBC Holdings                      HSBC Holdings
Verizon Communications    Verizon Communications
BNP Paribas                          BNP Paribas
Microsoft                              Microsoft
Name: name, dtype: object